## Training Notebook 

### The following cell is a way to get the utility script required for this notebook. 
Since IBM CPD SaaS doesn't have a filesystem, this is the only reliable way to get scripts on the cloud environment.

```
!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git
```

⚠️ Run the following cells only if you are executing on IBM CPD SaaS.

In [ ]:
#!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git

In [ ]:
#!mv MLOps-CPD MLOps_CPD

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install hyperopt ibm_aigov_facts_client ibm_watson_machine_learning ibm_watson_studio_pipelines lightgbm

In [ ]:
from botocore.client import Config
from ibm_botocore.client import Config
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.metrics import roc_auc_score,confusion_matrix,plot_confusion_matrix,plot_roc_curve,f1_score,auc,roc_curve,accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from ibm_aigov_facts_client import AIGovFactsClient, CloudPakforDataConfig
from ibm_watson_studio_pipelines import WSPipelines
from ibm_watson_machine_learning import APIClient
import warnings
import os, types
import pandas as pd
import numpy as np
import ibm_boto3
import json
import pickle
import matplotlib.pyplot as plt

from os.path import exists
if exists("utils/fs_utils.py") and exists("utils/catalog_utils.py"):
    from utils import fs_utils,catalog_utils
else:
    # If utils/fs_utils.py and utils/catalog_utils.py exist we assume that you are running on CPD SaaS
    # and will therefore import scripts from the freshly cloned repository
    from MLOps_CPD.utils import fs_utils, catalog_utils

warnings.filterwarnings("ignore")

try:
    import seaborn  as sns
except Exception as e:
    print(str(e) + " - Optional pluging skipped")
    

## Pipeline Params

In [ ]:
## REMOTE AUTH

TOKEN = os.getenv("USER_ACCESS_TOKEN")

CPD_HOST = os.getenv("CPD_HOST")
CPD_USERNAME = os.getenv("CPD_USERNAME")
CPD_PASSWORD = os.getenv("CPD_PASSWORD")

## LOCAL FS

path = os.getenv("path")

pipeline_filename = os.getenv("pipeline_filename")

train_data_filename = os.getenv("train_data_filename")
test_data_filename = os.getenv("test_data_filename")

### Construct paths
train_data_path = os.path.join(path, train_data_filename)
test_data_path = os.path.join(path, test_data_filename)

pipeline_path = os.path.join(path, pipeline_filename)

## WML / Factsheets

EXPERIMENT_NAME = os.getenv("EXPERIMENT_NAME")
MODEL_NAME = os.getenv("MODEL_NAME")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

SPACE_ID = os.getenv("SPACE_ID")
PROJECT_ID = os.getenv("PROJECT_ID")

## Utility Functions

In [ ]:
def check_for_file_in_filesystem(path):
    """
    Check existence of path in filesystem
    """
    if os.path.exists(path):
        return True
    else:
        print("File not found in specified path.")
        return False     

def load_model(key, filename):
    check_for_file_in_filesystem(filename)
    with open (filename,"rb") as f:
        pipeline = pickle.load(f)
    return pipeline

def load_data_from_filesystem(path):
    """
    Check existence of path in filesystem.
    If it does exist, loads csv via path
    If it does NOT exist, try to load data from Db2
    """
    body = check_for_file_in_filesystem(path)
    if body:
        suffix = path[-3:]
        # Check whether path ends on csv
        if suffix == "csv":
            gcf_df = pd.read_csv(path)
        else:
            with open(path) as f:
                gcf_df = pickle.load(f)

        return gcf_df
    else:
        print("\n")
        print(f"{path} file/path is probably not in project. Loading File from MLOps COS Bucket.")

        data_request = {
                'connection_name': """DB2_DATA""",
                'interaction_properties': {
                    'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
                }
            }

        gcf_df = read_data_from_db2(data_request)
        return gcf_df


def read_data_from_db2(data_request):
    """
    
    If load_data_from_filesystem fails, this method is executed.
    """
    read_client = itcfs.get_flight_client()
    DB2_DATA_data_request = {
        'connection_name': """DB2_DATA""",
        'interaction_properties': {
            'select_statement': 'SELECT * FROM "CUSTOMER_DATA"."GERMAN_CREDIT_RISK_TRAINING" FETCH FIRST 5000 ROWS ONLY'
        }
    }

    flightInfo = itcfs.get_flight_info(read_client, nb_data_request=data_request)

    df = itcfs.read_pandas_and_concat(read_client, flightInfo, timeout=240)
    return df

def save_data_in_filesystem(df,filename):
    """
    Save Data in Filesystem

    Passed filename should involve path

    """
    try:
        if filename[-3:] == "csv":
            df.to_csv(filename,index=False)
            print(f"File {filename} persisted successfully as csv")
        else:
            with open(filename, 'wb') as f:
                pickle.dump(df, f)
            print(f"File {filename} pickled successfully")
    except Exception as e:
        print(e)
        print(f"File serialization for {filename} failed")

## Load the Saved Transformer from IBM COS 

In [ ]:
pipeline = load_model(pipeline_path, pipeline_path)

## Instantiate FactSheets Client

In [ ]:
# Set-up Factsheets
cpd_creds = CloudPakforDataConfig(service_url=CPD_HOST, username=CPD_USERNAME, password=CPD_PASSWORD)
    
if SPACE_ID is None:
    # Only of SPACE_ID is unavailble, fallback to PROJECT_ID
    if PROJECT_ID is None:
        # Neither SPACE_ID nor PROJECT_ID available
        pass
    else:
        # PROJECT_ID available
        deploy_to = 'project'
        try:
            # Create new experiment
            facts_client = AIGovFactsClient(experiment_name=EXPERIMENT_NAME,container_type=deploy_to,container_id=PROJECT_ID,cloud_pak_for_data_configs=cpd_creds)
        except Exception as e:
            # Experiment already exists.. set as current experiment
            facts_client = AIGovFactsClient(experiment_name=EXPERIMENT_NAME,container_type=deploy_to,container_id=PROJECT_ID,cloud_pak_for_data_configs=cpd_creds,set_as_current_experiment=True)
        else:
            print(e)
else:
    # SPACE_ID is available
    deploy_to = 'space'
    try:
        # Create new experiment
        facts_client = AIGovFactsClient(experiment_name=EXPERIMENT_NAME,container_type=deploy_to,container_id=SPACE_ID,cloud_pak_for_data_configs=cpd_creds)
    except Exception as e:
        # Experiment already exists.. set as current experiment
        facts_client = AIGovFactsClient(experiment_name=EXPERIMENT_NAME,container_type=deploy_to,container_id=SPACE_ID,cloud_pak_for_data_configs=cpd_creds,set_as_current_experiment=True)
    else:
        print(e)
        
print(f"Setting up Factsheets in {deploy_to}")

## Load Train Data and Test Data 

In [ ]:
train_data = load_data_from_filesystem(train_data_path)
test_data = load_data_from_filesystem(test_data_path)
train_data.head()

## Load train and test set

In [ ]:
y_train = train_data['Risk']

y_test = test_data['Risk']

X_train = train_data.drop('Risk',axis=1)

X_test = test_data.drop('Risk',axis=1)

## Make validation set

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

## Instantiate a Classifier 

In [ ]:
from lightgbm import LGBMClassifier

model_lgb = LGBMClassifier(learning_rate=0.09,max_depth=5,random_state=42)

## Append the Pipeline

In [ ]:
estimator_step = ['model_lgb',model_lgb]
pipeline.steps.append(estimator_step)

In [ ]:
pipeline.steps[0][1].fit(X_tr)

## Baseline Model 

In [ ]:
# model_pipeline = pipeline.fit(X_train,y_train)
model_pipeline = pipeline.fit(X_tr,y_tr,model_lgb__verbose=5, model_lgb__eval_set=[(pipeline.steps[0][1].transform(X_val), y_val),(pipeline.steps[0][1].transform(X_tr), y_tr)])

In [ ]:
from lightgbm import plot_metric
plot_metric(pipeline.steps[1][1])

## Log the Train and Val loss

In [ ]:
val_loss = pipeline.steps[1][1].evals_result_['valid_0']
train_loss = pipeline.steps[1][1].evals_result_['valid_1']

## Save train and val loss to COS

In [ ]:
val_loss_path = os.path.join(path, 'val_loss.pkl')
save_data_in_filesystem(val_loss, val_loss_path)

train_loss_path = os.path.join(path, 'train_loss.pkl')
save_data_in_filesystem(train_loss, train_loss_path)

## Check if the files are copied in COS


In [ ]:
files_copied_in_cos = check_for_file_in_filesystem(val_loss_path) and check_for_file_in_filesystem(train_loss_path)
files_copied_in_cos

## Baseline Results of the Model

In [ ]:
predictions = pipeline.predict(X_test)

print(roc_auc_score(y_test,predictions))

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=1)
scores = np.mean(cross_val_score(pipeline,X_train,y_train, cv=cv, n_jobs=-1,scoring='roc_auc'))
print(f"The Cross Validated AUC_ROC Score is {scores}")

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(pipeline.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(pipeline.score(X_test, y_test)))

In [ ]:
df_cm = confusion_matrix(y_test,predictions)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in df_cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in df_cm.flatten()/np.sum(df_cm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2,v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

try:
    sns.heatmap(df_cm, annot=labels, fmt='', cmap='Blues')
except Exception as e:
    print(str(e) + " - Seaborn missing, skipping optional heatmap plot.")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, predictions)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12

plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

print("\n")
print ("Area Under Curve: %.2f" %auc(fpr, tpr))
print("\n")

## Serialize Model locally

In [ ]:
model_path = os.path.join(path, MODEL_NAME+".pkl")
model_path

In [ ]:
with open(model_path,'wb') as f:
    pickle.dump(model_pipeline,f)

In [ ]:
cpd_ver = facts_client.get_cpd_version()[:3]

In [ ]:
WML_CREDENTIALS = {
   "token": TOKEN,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

In [ ]:
wml_client = APIClient(WML_CREDENTIALS)

## Save and Log Models in AI Factsheets.

In [ ]:
# TODO: Rework this without helper scripts

## Save Params in WS Pipeline

In [ ]:
train_params = {}
train_params['auc_roc'] = float(auc(fpr, tpr))
train_params['training_done'] = True
train_params['model_name'] = MODEL_NAME
train_params['model_path'] = model_path
train_params['deployment_name'] = DEPLOYMENT_NAME
# train_params['model_id'] = model_id
train_params['project_id'] = project_id if not None else None
# train_params['model_pipeline'] = "/home/wsuser/work/model_pipeline.pkl"

pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(train_params)